<a href="https://colab.research.google.com/github/Zardian18/Cybersec_assistant/blob/master/CyberAssistant_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00


In [3]:
!pip install -U PyPDF2
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.4 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

dataset = load_dataset("ahmed000000000/cybersec")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['INSTRUCTION', 'RESPONSE'],
        num_rows: 12408
    })
})

In [7]:
import pandas as pd

In [8]:
train_ds = pd.DataFrame(dataset["train"])
train_ds

,INSTRUCTION,RESPONSE
0,Can you explain how to use WNetAddConnection t...,To use WNetAddConnection to map a network shar...
1,Compare and contrast the security features of ...,All three web servers have different security ...
2,Define 'Clickjacking' and describe how it can ...,Clickjacking is a type of attack where a malic...
3,Define 'Computed IOCs' and give examples of ea...,Computed IOCs refer to Indicators of Compromis...
4,Define 'Containment and intelligence developme...,Containment and intelligence development is a ...
...,...,...
12403,What design issue can be exploited to imperson...,The design issue that can be exploited to impe...
12404,Explain how responder-like activity can be det...,Responder-like activity in a network can be de...
12405,What is osquery and how can it be used to dete...,osquery is a tool that can be used to detect W...
12406,Explain how to detect the Netsh persistence te...,"To detect the Netsh persistence technique, fil..."


In [9]:
text_data= train_ds["RESPONSE"]
text_data

0        To use WNetAddConnection to map a network shar...
1        All three web servers have different security ...
2        Clickjacking is a type of attack where a malic...
3        Computed IOCs refer to Indicators of Compromis...
4        Containment and intelligence development is a ...
                               ...                        
12403    The design issue that can be exploited to impe...
12404    Responder-like activity in a network can be de...
12405    osquery is a tool that can be used to detect W...
12406    To detect the Netsh persistence technique, fil...
12407    The Windows event ID that is logged when an ad...
Name: RESPONSE, Length: 12408, dtype: object

In [10]:
text_data.to_csv('text.txt', index=False, header=False)

In [11]:
text_data = text_data.tolist()
text_data[:5]

['To use WNetAddConnection to map a network share, first make sure that the computer has access to the network share and that the appropriate network resource is available. Then, use the WNetAddConnection function to establish a connection to the network resource by specifying the path to the share as well as the user name and password necessary to access the share. Once connected, you can then access files and resources on the network share as if they were local to the machine. Note that the WNetAddConnection function should only be used within trusted environments, as using it to establish connections to untrusted or malicious network resources can pose a serious security threat to the system.',
 'All three web servers have different security features that set them apart. Apache is known for its mod_security module, which provides advanced security features like HTTP filtering and Real-Time Block Lists to protect against SQL injection, cross-site scripting, and other attacks. IIS has

In [12]:
!mkdir output

## Trainig

In [13]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Set the path to your custom dataset
text_data_path = "/content/text.txt"

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can use other variants like "gpt2-medium" or "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load your custom dataset
with open(text_data_path, "r", encoding="utf-8") as file:
    text_data = file.read()

# Tokenize the dataset
tokenized_data = tokenizer.encode(text_data, return_tensors="pt")

# Create a TextDataset and DataCollatorForLanguageModeling
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=text_data_path,
    block_size=128,  # Adjust the block size as needed
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True if your dataset includes masked language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/output",
    overwrite_output_dir=True,
    num_train_epochs=5,  # Adjust the number of epochs as needed
    per_device_train_batch_size=8,  # Adjust batch size as needed
    save_steps=5000,  # Save model checkpoints every N steps
    save_total_limit=3,
)

# Create Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1527802 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# !pip uninstall transformers
# !pip uninstall accelerate

In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [16]:
trainer.train()

Step,Training Loss
500,2.715500
1000,2.488100
1500,2.393900
2000,2.240500
2500,2.208600
3000,2.172200
3500,2.066400
4000,2.073200
4500,2.039800
5000,1.961800


TrainOutput(global_step=7460, training_loss=2.1378965976091235, metrics={'train_runtime': 2292.0201, 'train_samples_per_second': 26.036, 'train_steps_per_second': 3.255, 'total_flos': 3898150502400000.0, 'train_loss': 2.1378965976091235, 'epoch': 5.0})

In [17]:
trainer.push_to_hub(commit_message="Training completed!")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

events.out.tfevents.1710248455.9e467a4cc2a2.1252.0:   0%|          | 0.00/8.10k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zardian/output/commit/daa03b55f8504840527cb04e87fdfc32ffef58da', commit_message='Training completed!', commit_description='', oid='daa03b55f8504840527cb04e87fdfc32ffef58da', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
# from transformers import pipeline
# model_id = "Zardian/output"
# generator = pipeline("text-generation", model=model_id)

OSError: Can't load tokenizer for 'Zardian/output'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Zardian/output' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.

In [25]:
input_txt ="What is ransomware?"

In [23]:
import torch

# Check if GPU is available and choose the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
# input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 100
choices_per_step = 5

In [28]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is ransomware? It is a type of cyber attack that targets computer systems and networks. It is a type of ransomware that encrypts files and demands a ransom for their release. The ransomware is a type of ransomware that encrypts files and demands a ransom for their release. The purpose of ransomware is to disrupt, damage, or gain unauthorized access to computer systems and networks. It is a significant concern in cybersecurity as it is a common attack vector for cybercriminals."
"Ransomware refers to a type of


In [ ]:
import locale
print(locale.getpreferredencoding())

In [38]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [41]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 1.5 MB/s eta 0:00:00


In [47]:
import gradio as gr

In [54]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [52]:
def generate_text(inp):
  input_ids = tokenizer.encode(inp, return_tensors="pt")
  beam_output = model.generate(input_ids, max_len=150, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
  output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  return ".".join(output.split(".")[:-1])+ "."

In [65]:
def generate_text(inp):
  input_ids = tokenizer(inp, return_tensors="pt")["input_ids"].to(device)
  output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
  return tokenizer.decode(output[0])

In [63]:
generate_text("what is ransomware")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"What is ransomware?\nRansomware is a type of cyber attack that targets computer systems and networks. It is designed to steal sensitive information, such as passwords, credit card numbers, or personal data, from victims' devices or networks. The impact of ransomware on cybersecurity is significant as it can lead to financial loss, reputational damage, and disruption of critical services. It is crucial for organizations to implement robust security measures, such as encryption and regular system updates, to protect against ransomware threats and ensure the confidentiality and"

In [66]:
import gradio as gr
iface = gr.Interface(
    fn=generate_text,
    inputs="textbox",
    outputs="text",
    title="CYBERSEC_ASSISTANT",
    description="null"
)
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://29ad28028f7e41e2e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
